In [90]:
import pandas as pd
import numpy as np
from preprocess import scale
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB

In [83]:
url = 'https://raw.githubusercontent.com/probml/pmtkdata/master/spamData/spam.data'
data = pd.read_table(url, header = None, sep = ' ')

In [84]:
y = data[57].values
X = data[range(57)].values
del data

In [85]:
train = np.random.choice(len(y), replace = False, size = 3065)
test = np.ones(len(y), dtype=bool)
test[train] = 0

In [86]:
train_y = y[train]
test_y = y[test]
train_X = X[train,]
test_X = X[test,]
del X
del y

In [129]:
import generative
reload(generative)
import nb
reload(nb)

clf = nb.Bernoulli()
clf.fit(1 * (train_X > 0), train_y)
y_hat = clf.predict(1 * (test_X > 0))
print '%0.3f' % np.mean(np.mean(y_hat != test_y))

0.111


In [112]:
import htwt
import nb
reload(nb)

X, y = htwt.get()

clf = nb.Gaussian()
clf = GaussianNB()
clf.fit(X, y)
y_hat = clf.predict(X)
print '%0.3f' % np.mean(np.mean(y_hat != y))

0.124


In [107]:
def test(transform, C):
    clf = LogisticRegression(C = C)
    clf.fit(transform(train_X), train_y)
    result = lambda X, y: '%0.3f' % np.mean(np.mean(clf.predict(transform(X)) != y))
    print result(train_X, train_y), result(test_X, test_y)

for C in np.arange(0.5, 2, 0.5):
    print C
    test(lambda X: X, C)
    test(lambda X: scale(X), C)
    test(lambda X: np.log(X + 0.1), C)
    test(lambda X:  1 * (X > 0), C)

0.5
0.070 0.070
0.074 0.070
0.054 0.049
0.067 0.066
1.0
0.069 0.067
0.070 0.068
0.054 0.048
0.067 0.066
1.5
0.069 0.070
0.071 0.068
0.054 0.049
0.066 0.066
